In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = read_csv('https://raw.githubusercontent.com/datagy/mediumdata/master/AAPL.csv', parse_dates=['Date'])

print(df.head())

NameError: name 'read_csv' is not defined

In [8]:
url = 'https://raw.githubusercontent.com/tvelichkovt/PyTorch/master/flights.csv'
df = pd.read_csv(url)
print(df.head(5))

ParserError: Error tokenizing data. C error: Expected 1 fields in line 98, saw 2
